Importing libs for analysis.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Importing the data.

In [3]:
data = pd.read_csv("titanic_train.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Cleaning the data.

In [4]:
    data.drop(['Cabin','Ticket'], axis=1, inplace=True)
    def name_extract(word):
        return word.split(',')[1].split('.')[0].strip()
    salutation = pd.DataFrame({'Salutation': data['Name'].apply(name_extract)})
    def group_salutation(old_salutation):
        if old_salutation == 'Mr':
            return('Mr')
        else:
            if old_salutation == 'Mrs':
                return('Mrs')
            else:
                if old_salutation == 'Master':
                    return('Master')
                else: 
                    if old_salutation == 'Miss':
                        return('Miss')
                    else:
                        if old_salutation == 'Master':
                            return('Master')
                        else:
                            return('Others')
    salutation = pd.DataFrame({'Salutation': salutation['Salutation'].apply(group_salutation)})
    data = pd.merge(data, salutation, left_index = True, right_index = True)
    table = data.pivot_table(values='Age', index=['Salutation'], columns=['Pclass', 'Sex'], aggfunc=np.mean)
    def fage(x):
        return table[x['Pclass']][x['Sex']][x['Salutation']]
    data['Age'].fillna(data[data['Age'].isnull()].apply(fage, axis=1), inplace=True)
    data.dropna(inplace=True)
    data.ix[data.Fare > 200, 'Fare'] = 200
    data['Family_size'] = data['SibSp'] + data['Parch'] + 1
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    data.Embarked = le.fit_transform(data.Embarked)
    data.Salutation = le.fit_transform(data.Salutation)
    data.Sex = le.fit_transform(data.Sex)
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder(categorical_features='all')
    data_encoded = enc.fit_transform(data[['Embarked','Salutation','Pclass']]).toarray()
    data_encoded = pd.DataFrame(data_encoded)
    data = pd.merge(data, data_encoded, left_index = True, right_index = True)

In [5]:
X = data.drop(['PassengerId','Survived', 'Name', 'Pclass', 'Embarked', 'Salutation', 'Parch', 'SibSp', 7, 5, 10, 0, 4, 9], axis=1)
y = data.ix[:, 'Survived']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
clf = DecisionTreeClassifier()
cross_val_score(clf, X, y, cv=10).mean()

In [ ]:
from sklearn.grid_search import GridSearchCV

criterion = ['gini', 'entropy']
splitter = ['best', 'random']
max_depth = np.arange(1,10)
min_samples_leaf = np.arange(1,20,2)
min_samples_split = np.arange(1,20,2)

parameters = {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth, 
              'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split}
clf = DecisionTreeClassifier(random_state=0)
#clf = GridSearchCV(clf, parameters, cv=10)
#clf.fit(X, y)

#print clf.best_score_
#print clf.best_params_

# Test set

In [6]:
data_test = pd.read_csv("titanic_test.csv")
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
    data_test.drop(['Cabin','Ticket'], axis=1, inplace=True)
    def name_extract(word):
        return word.split(',')[1].split('.')[0].strip()
    salutation = pd.DataFrame({'Salutation': data_test['Name'].apply(name_extract)})
    def group_salutation(old_salutation):
        if old_salutation == 'Mr':
            return('Mr')
        else:
            if old_salutation == 'Mrs':
                return('Mrs')
            else:
                if old_salutation == 'Master':
                    return('Master')
                else: 
                    if old_salutation == 'Miss':
                        return('Miss')
                    else:
                        if old_salutation == 'Master':
                            return('Master')
                        else:
                            return('Others')
    salutation = pd.DataFrame({'Salutation': salutation['Salutation'].apply(group_salutation)})
    data_test = pd.merge(data_test, salutation, left_index = True, right_index = True)
    #print data.shape
    #print data.isnull().sum()
    #print data.Salutation.value_counts()
    data_test['Age'].fillna(data_test[data_test['Age'].isnull()].apply(fage, axis=1), inplace=True)
    #print data[data.isnull().any(axis=1)]
    data_test['Fare'].fillna(data_test['Fare'].mean(), inplace=True)
    data_test['Age'].fillna(data_test['Age'].mean(), inplace=True)
    #print data.shape
    #print data.isnull().sum()
    data_test.dropna(inplace=True)
    #print data.shape
    data_test.ix[data_test.Fare > 200, 'Fare'] = 200
    data_test['Family_size'] = data_test['SibSp'] + data_test['Parch'] + 1
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    data_test.Embarked = le.fit_transform(data_test.Embarked)
    data_test.Salutation = le.fit_transform(data_test.Salutation)
    data_test.Sex = le.fit_transform(data_test.Sex)
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder(categorical_features='all')
    data_encoded = enc.fit_transform(data_test[['Embarked','Salutation','Pclass']]).toarray()
    data_encoded = pd.DataFrame(data_encoded)
    data_test = pd.merge(data_test, data_encoded, left_index = True, right_index = True)

In [8]:
X_test = data_test.drop(['PassengerId', 'Name', 'Pclass', 'Embarked', 'Salutation',  'Parch', 'SibSp', 7, 5, 10, 0, 4, 9], axis=1)

In [8]:
clf = DecisionTreeClassifier(min_samples_split=5, splitter='best', criterion='entropy', max_depth=6, min_samples_leaf=1)
clf.fit(X, y)

NameError: name 'DecisionTreeClassifier' is not defined

In [ ]:
predicted = pd.Series(clf.predict(X_test))
solution = pd.concat([data_test['PassengerId'], predicted], axis=1)
solution.columns = ['PassengerId','Survived']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

cv = 10

clf = DecisionTreeClassifier(random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print "Decision Tree Score: %.3f" % scores.mean()
print "Decision Tree Variance: %.4f \n" % scores.std()

clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print "Random Forest Score: %.3f" % scores.mean()
print "Random Forest Variance: %.4f \n" % scores.std()

clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print "Extremely Randomized Trees Score: %.3f" % scores.mean()
print "Extremely Randomized Trees Variance: %.4f \n" % scores.std()

clf = AdaBoostClassifier(n_estimators=100, random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print "AdaBoost Score: %.3f" % scores.mean()
print "AdaBoost Variance: %.4f \n" % scores.std()

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print "Gradient Tree Boosting Score: %.3f" % scores.mean()
print "Gradient Tree Boosting Variance: %.4f \n" % scores.std()

In [ ]:
#criterion = ['friedman_mse', 'mse']
learning_rate = np.arange(0.1,1,.1)
loss = ['deviance', 'exponential']
max_depth = np.arange(1,10)
min_samples_split = np.arange(1,10,2)

parameters = {'learning_rate': learning_rate, 'loss': loss, "max_depth": max_depth, "min_samples_split": min_samples_split}
clf = RandomForestClassifier(random_state=0)
clf = GridSearchCV(clf, parameters, cv=10)
clf.fit(X, y)

print clf.best_score_
print clf.best_params_

In [ ]:
criterion = ['friedman_mse', 'mse']
learning_rate = np.arange(0.1,1,.1)
loss = ['deviance', 'exponential']
max_depth = np.arange(1,10)
min_samples_split = np.arange(1,10,2)

parameters = {'learning_rate': learning_rate, 'loss': loss, "max_depth": max_depth, "min_samples_split": min_samples_split}
clf = RandomForestClassifier(random_state=0)
clf = GridSearchCV(clf, parameters, cv=10)
clf.fit(X, y)

print clf.best_score_
print clf.best_params_

In [ ]:
clf = RandomForestClassifier(min_samples_split=5, criterion='entropy', max_depth=6, min_samples_leaf=1)
clf.fit(X, y)

In [ ]:
clf = GradientBoostingClassifier()
clf.fit(X, y)

In [ ]:
clf.feature_importances_

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_order = X.columns[indices]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, feature_order[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(X.shape[1]), feature_order, rotation='vertical')
plt.xlim([-1, X.shape[1]])
plt.show()

In [9]:
import xgboost as xgb

In [12]:
def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [10]:
from sklearn.cross_validation import train_test_split
train_x, eval_x, train_y, eval_y = train_test_split(X, y, test_size=0.25, random_state=14)

In [11]:
params = {"objective": "reg:linear",
            "eta": 0.02,
            "max_depth": 10,
            "subsample": 0.5,
            "colsample_bytree": 0.7,
            "silent": 1
             }
num_trees = 2000
dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(eval_x, eval_y)
dtest = xgb.DMatrix(X_test)
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=75, feval=rmspe_xg, verbose_eval=True)
prediction = gbm.predict(dtest)

NameError: name 'rmspe_xg' is not defined

In [ ]:
predicted = pd.Series(clf.predict(X_test))
solution = pd.concat([data_test['PassengerId'], predicted], axis=1)
solution.columns = ['PassengerId','Survived']

In [ ]:
solution.to_csv('solution.csv', index=False)